In [1]:
import cv2
import numpy as np

# Yolo 로드

In [2]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [3]:
# 이미지 가져오기
img = cv2.imread("abc.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Detecting objects
320 × 320 : 작고 정확도는 떨어지지 만 속도 빠름

609 × 609 : 정확도는 더 높지만 속도 느림

416 × 416 : 중간

outs는 감지 결과이다. 탐지된 개체에 대한 모든 정보와 위치를 제공한다.

In [4]:
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# 정보를 화면에 표시

결과 화면에 표시 / 신뢰도, 신뢰 임계값  계산 (이 부분이 완전 이해가 안된다 ) 

신뢰도가 0.5 이상이라면 물체가 정확히 감지되었다고 간주한다. 아니라면 넘어감.. 

임계값은 0에서 1사이의 값을 가지는데

1에 가까울수록 탐지 정확도가 높고 , 0에 가까울수록 정확도는 낮아지지만 탐지되는 물체의 수는 많아진다.

In [5]:
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # 좌표
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# 노이즈제거
같은 물체에 대한 박스가 많은것을 제거

Non maximum suppresion이라고 한답니다.

In [6]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# 화면에 표시
마지막으로 모든 정보를 추출하여 화면에 표시합니다. 

Box : 감지된 개체를 둘러싼 사각형의 좌표

Label : 감지된 물체의 이름

Confidence : 0에서 1까지의 탐지에 대한 신뢰도

In [7]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
# cv2.imshow("Image", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
 cv2.imwrite('Output_.jpg', img) # opencv로 저장 

True